In [1]:
# IMPORTING EVERYTHING I COULD POSSIBLY NEED #

%matplotlib inline
import numpy as np
import pandas as pd
import re
from nltk.tokenize import TreebankWordTokenizer

import matplotlib
import sklearn
from IPython.core.display import display, HTML

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

get_ipython().run_line_magic('matplotlib', 'inline')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid", font_scale=1.9)

# TO WORK WITH
import pandas as pd
import numpy as np
from numpy import set_printoptions

# HIDE WARNINGS
import warnings
warnings.filterwarnings('ignore')

# PREPROCESSING & MODEL SELECTION
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import randint
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeCV, LassoCV, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import SCORERS
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

# print(SCORERS.keys())

# PLOTTING
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook")
get_ipython().run_line_magic('matplotlib', 'inline')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn import tree
from graphviz import Source
from matplotlib.pylab import rcParams
import matplotlib.lines as mlines
from yellowbrick.classifier import PrecisionRecallCurve
from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import InterclusterDistance
import plotly.express as px
import scipy.cluster.hierarchy as sch
from sklearn.metrics import classification_report


# STANDARD MODELS
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# ENSEMBLE
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

# XGBOOST
from xgboost import XGBClassifier
import xgboost as xgb

# CLUSTERING
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

# PICKLE
from pickle import dump
from pickle import load

In [2]:
def recommendations(coefs, airbnb, X_wnei, data, recom):
    '''
    Makes recommendations based on the specific Airbnb's characteristics and the important features recognized by
    the best model in best_models()

    '''
#   X columns and model's nonzero column indeces
#     X_wnei_cols = X_wnei.columns
#     nonz = np.nonzero(coefs)[0]

#   Names of important columns
#     imp_cols = []
#     for col in nonz:
#         imp_cols.append(X_wnei_cols[col])

## This commented out part deletes everything having to do with reviews and id,
## while we are already dropping them in best_models()

# #   Airbnb's areas of improvement according to the model's nonzero columns
#     for_improvement = []
#     ind = 0
#     for col in nonz:
#         if col != 0:
#             if "review" not in X_wnei_cols[col] and "reviews" not in X_wnei_cols[col]:
#                 name = imp_cols[ind]
#                 val  = airbnb[0][col]
#                 mean = data.iloc[:,col].mean()
#                 if val < mean:
#                     for_improvement.append(name)
#         ind += 1


#   Airbnb's areas of improvement according to the model's nonzero columns
#     for_improvement = []
#     ind = 0
#     for col in nonz:
#         name = imp_cols[ind]
#         val  = airbnb[0][col]
#         mean = data.iloc[:,col].mean()
#         if val < max(data.iloc[:,col]):
#             print(val,mean)
#     #         print(1)
#             if coefs[col] < 0:
#                 a=-1
#             else:
#                 a=1
#     #         print(2)
#             if a*val < 0:
#                 sent = recom.iloc[1, col]
#                 for_improvement.append(sent)
#             elif a*val > 0:
#                 sent = recom.iloc[0, col]
#                 for_improvement.append(sent)

#         ind += 1


    X_wnei_cols = X_wnei.columns

    for_improvement = []
    for col in range(1,len(X_wnei.columns)):
        if coefs[col] > 0 :
            if airbnb[0][col] < data.iloc[:,col].mean():
                if "cancellations" not in recom.iloc[0, col] and ("You should be more moderate on cancellations" not in for_improvement and "You should be less strict on cancellations" not in for_improvement):
                    if "messages" not in recom.iloc[0, col] and ("You should answer messages sooner" not in for_improvement):
                        sent = recom.iloc[0, col]
                        for_improvement.append(sent)
        elif coefs[col] < 0:
            if airbnb[0][col] > data.iloc[:,col].mean():
                if "cancellations" not in recom.iloc[1, col] and ("You should be more moderate on cancellations" not in for_improvement and "You should be less strict on cancellations" not in for_improvement):
                    if "messages" not in recom.iloc[1, col] and ("You should answer messages sooner" not in for_improvement):
                        sent = recom.iloc[1, col]
                        for_improvement.append(sent)

#     print(for_improvement)
    # If there are improvements to make...
    if len(for_improvement) != 0:
    #   Concatenating messages
        mess1 = "In order for your Airbnb to be truly competitive, make sure you do the following things: "
        mess2 = "\n".join(for_improvement)
        message = mess1 + "\n" + mess2

        return print(message)
    # If there are NO improvements to make...
    else:
        return print("Your Airbnb is very competitive compared to all the other Airbnbs in Boston!")


In [3]:
def best_models(data):
    '''
    Implements Ridge, Lasso and ElasticNet to determine best model, based on inputted data.

    '''
    MSEs = []


    data=data.dropna()

    data=data[data["number_of_reviews"]>=1]
    data=data[data["guests_included"]>=1]

    data['price_per_person'] = data['price_per_night']/(data['guests_included']+data['extra_people'])

    data['output'] = (data['number_of_reviews']*data['review_scores_rating'])/data['availability_365']

    data = data.drop(["property_type","room_type","description","house_rules", "amenities", "id",
    "review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness",
    "review_scores_checkin", "review_scores_communication", "review_scores_value", "number_of_reviews",
                      "reviews_per_month", "availability_365", "neighbourhood_cleansed","bathrooms",
                      "host_is_superhost", "accommodates", "bedrooms","beds", "guests_included","price_per_night",
                      "extra_people", "host_is_superhost", "Kitchen_boolean", "Gym_bool", "Elevator_in_building_bool",
                      "Clothes_Washer_bool", "Internet_bool", "price_per_person"],axis=1)

    scaler=MinMaxScaler(feature_range=(0,1))


    data[['security_deposit']] = scaler.fit_transform(data[['security_deposit']])
    data[['cleaning_fee']] = scaler.fit_transform(data[['cleaning_fee']])
    data[['host_response_rate']] = scaler.fit_transform(data[['host_response_rate']])
#     data[['price_per_person']] = scaler.fit_transform(data[['price_per_person']])

    data.to_numpy()
    data=pd.DataFrame(data)

    data=data.dropna()

    data['output'][data['output'] > 260] = 260

    y = data["output"] # Target variable (price)
    X_wnei = data.drop(["output"],axis=1)

    # Creating new DF without neighborhood names
    X_wnei.to_csv("X_wnei.csv", index=False)
    data.to_csv("data.csv", index=False)

    #########################################################################################################
    # Ridge Regression

    kfold=KFold(n_splits=10, random_state=7)

    model=Ridge()
    scoring = "neg_mean_squared_error"

    results=cross_val_score(model, X_wnei, y, cv=kfold, scoring=scoring)
    clf = model.fit(X_wnei, y)
    MSEs.append(("Ridge Regression", results.mean(), clf.coef_))


    #########################################################################################################
    # Lasso Regression

    kfold=KFold(n_splits=10, random_state=7)

    model=Lasso()
    scoring = "neg_mean_squared_error"

    results=cross_val_score(model, X_wnei, y, cv=kfold, scoring=scoring)
    clf = model.fit(X_wnei, y)
    MSEs.append(("Lasso Regression", results.mean(), clf.coef_))

    #########################################################################################################
    # Elastic Net Regression

    kfold=KFold(n_splits=10, random_state=7)

    model=ElasticNet()
    scoring = "neg_mean_squared_error"

    results=cross_val_score(model, X_wnei, y, cv=kfold, scoring=scoring)
    clf = model.fit(X_wnei, y)
    MSEs.append(("ElasticNet Regression", results.mean(), clf.coef_))
#     print(MSEs)

    #########################################################################################################

    return (min(MSEs, key = lambda t: t[1])[0], X_wnei, y, min(MSEs, key = lambda t: t[1])[2], data)



#############    MAIN

In [4]:
############# MAIN

data = pd.read_csv("listings_8.csv")
recom = pd.read_csv("recom.csv")

best_model = best_models(data)

method = best_model[0]
X_wnei = best_model[1]
y = best_model[2]
coefs = best_model[3].reshape(33,1)
data = best_model[4]


# Using an airbnb from our data so that we don't have to create one manually
# If you want to change the airbnb, chnage the iloc index to another one (in our data range)

# Example of airbnb that doesn't need any improvements according to our best model
print("\nExample of airbnb that needs improvements according to our best model")
print("-")
airbnb = pd.read_csv("X_wnei.csv").iloc[1345].values.reshape(1,-1)
recommendations(coefs, airbnb, X_wnei, data, recom)
print("____________________________________________________________________________________\n")

# Example of airbnb that needs improvements according to our best model
print("\nExample of airbnb that needs improvements according to our best model")
print("-")
airbnb = pd.read_csv("X_wnei.csv").iloc[151].values.reshape(1,-1)
recommendations(coefs, airbnb, X_wnei, data, recom)
print("____________________________________________________________________________________\n")



# Good ones to showcase!
#
# airbnb = pd.read_csv("X_wnei.csv").iloc[1345].values.reshape(1,-1) -> Nothing to improve
# airbnb = pd.read_csv("X_wnei.csv").iloc[1].values.reshape(1,-1) -> A lot to improve
#
# 


Example of airbnb that needs improvements according to our best model
-
Your Airbnb is very competitive compared to all the other Airbnbs in Boston!
____________________________________________________________________________________


Example of airbnb that needs improvements according to our best model
-
In order for your Airbnb to be truly competitive, make sure you do the following things: 
You should allow 24-hours check-in
You should install smoke detectors
You should remove/reduce the cleaning fee
You should allow instant booking
____________________________________________________________________________________



In [59]:
data = pd.read_csv("listings_8.csv")
data[["security_deposit"]]

,security_deposit
0,0
1,200
2,150
3,100
4,200
...,...
2763,0
2764,0
2765,0
2766,0


In [60]:
length = data[["security_deposit"]].shape[0]
length

2768

In [61]:
# data[["security_deposit"]][length] = 100
sec= data[["security_deposit"]].copy()
df2 = pd.DataFrame([[100]], columns=['security_deposit'])
sec = sec.append(df2)
sec

,security_deposit
0,0
1,200
2,150
3,100
4,200
...,...
2764,0
2765,0
2766,0
2767,100


In [46]:
sec
scaler=MinMaxScaler(feature_range=(0,1))
sec
sec = scaler.fit_transform(sec)    
sec

,security_deposit
0,0
1,200
2,150
3,100
4,200
...,...
2764,0
2765,0
2766,0
2767,100


,security_deposit
0,0
1,200
2,150
3,100
4,200
...,...
2764,0
2765,0
2766,0
2767,100


array([[0.        ],
       [0.21052632],
       [0.15789474],
       ...,
       [0.        ],
       [0.10526316],
       [0.10526316]])

In [49]:
# sec=sec[["security_deposit"]]
security_deposit = sec[-1:]

In [51]:
float(security_deposit)

0.10526315789473684

In [58]:


data["host_response_rate"].unique()

array([  0,  99,  98,  97,  96,  95,  94,  93,  92,  91,  90,  89,  88,
        86,  85,  84,  83,  82,  81,  80,  79,  78,  77,  75,  73,  71,
        70,   6,  69,  68,  67,  64,  61,  60,  57,  56,  50,  43,  40,
        38,  36,  33,  31,  30,  25,  19,  17,  14,  10, 100])